# Spark Learning Note - Table Joins

Jia Geng | gjia0214@gmail.com

In [1]:
# check java version 
# use sudo update-alternatives --config java to switch java version if needed.
!java -version

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~19.10-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)


In [3]:
from pyspark.sql.session import SparkSession

spark = SparkSession.builder.appName('TableJoinExamples').getOrCreate()
spark

## Join Operations

Join operation: `left.xJoin(right)` is to bring left and right tables together by one or more keys. Common join types include:
- **inner joins**: keep rows with from both tables keys exist in both left&right table
- **outer joins**: keep rows from both tables with keys exist in either left or right table
- **left outer joins**: keep rows from both tables with keys exist in left table
- **right outer joins**: keep rows from both tables  with keys exist in the right table
- **left semi joins**: keep rows from only left tables with keys exist in the left&right table. Result only contain columns from left table.
- **left anti joins**: keep rows from only left tables with keys NOT exist in the right table,  Result only contain columns from left table.
- **natural joins**: join by matching the columns between two datasets with the same names
    - be careful when using this as column name could be the same but indicating different things!!
- **cross joins**: match every row in the left table with every row in the right table

In spark:
```
df_left.join(df_left, on=, how=)
```

In [24]:
# set up the example
person_data = [(0, 'Bill', 0, [100]), (1, 'Matt', 1, [500, 250, 100]), (2, 'Zack', 1, [250, 100])]  
person_coln = ['id', 'name', 'graduate_program', 'status']
person = spark.createDataFrame(person_data, person_coln)

program_data = [(0, 'MS', 'ACE', 'UC'), (2, 'MS', 'ECCS', 'UC'), (1, 'PhD', 'ECCS', 'UC')]
program_coln = ['id', 'degree', 'department', 'school']
program = spark.createDataFrame(program_data, program_coln)

status_data = [(500, 'VP'), (250, 'PMC Member'), (100, 'Contributor')]
status_coln = ['id', 'status']
status = spark.createDataFrame(status_data, status_coln)

person.show()
program.show()
status.show()

+---+----+----------------+---------------+
| id|name|graduate_program|         status|
+---+----+----------------+---------------+
|  0|Bill|               0|          [100]|
|  1|Matt|               1|[500, 250, 100]|
|  2|Zack|               1|     [250, 100]|
+---+----+----------------+---------------+

+---+------+----------+------+
| id|degree|department|school|
+---+------+----------+------+
|  0|    MS|       ACE|    UC|
|  2|    MS|      ECCS|    UC|
|  1|   PhD|      ECCS|    UC|
+---+------+----------+------+

+---+-----------+
| id|     status|
+---+-----------+
|500|         VP|
|250| PMC Member|
|100|Contributor|
+---+-----------+



In [43]:
# inner join person and program on program id
# it kept the id column from both table
# id from person is the person id
# id from program is the program id
# they are different!
person.join(program, person['graduate_program'] == program['id'], 'inner').show()
person.join(program, person['graduate_program'] != program['id'], 'inner').show()

+---+----+----------------+---------------+---+------+----------+------+
| id|name|graduate_program|         status| id|degree|department|school|
+---+----+----------------+---------------+---+------+----------+------+
|  0|Bill|               0|          [100]|  0|    MS|       ACE|    UC|
|  1|Matt|               1|[500, 250, 100]|  1|   PhD|      ECCS|    UC|
|  2|Zack|               1|     [250, 100]|  1|   PhD|      ECCS|    UC|
+---+----+----------------+---------------+---+------+----------+------+

+---+----+----------------+---------------+---+------+----------+------+
| id|name|graduate_program|         status| id|degree|department|school|
+---+----+----------------+---------------+---+------+----------+------+
|  0|Bill|               0|          [100]|  2|    MS|      ECCS|    UC|
|  0|Bill|               0|          [100]|  1|   PhD|      ECCS|    UC|
|  1|Matt|               1|[500, 250, 100]|  0|    MS|       ACE|    UC|
|  1|Matt|               1|[500, 250, 100]|  2|   

In [29]:
# outer join person and program on program id
# if id can not be matched, missing side will be null
person.join(program, person['graduate_program'] == program['id'], 'outer').show()

+----+----+----------------+---------------+---+------+----------+------+
|  id|name|graduate_program|         status| id|degree|department|school|
+----+----+----------------+---------------+---+------+----------+------+
|   0|Bill|               0|          [100]|  0|    MS|       ACE|    UC|
|   1|Matt|               1|[500, 250, 100]|  1|   PhD|      ECCS|    UC|
|   2|Zack|               1|     [250, 100]|  1|   PhD|      ECCS|    UC|
|null|null|            null|           null|  2|    MS|      ECCS|    UC|
+----+----+----------------+---------------+---+------+----------+------+



In [32]:
# left/right outer join person and program on program id
# only keep the rows from the left/right table
person.join(program, person['graduate_program'] == program['id'], 'left_outer').show()
person.join(program, person['graduate_program'] == program['id'], 'right_outer').show()

+---+----+----------------+---------------+---+------+----------+------+
| id|name|graduate_program|         status| id|degree|department|school|
+---+----+----------------+---------------+---+------+----------+------+
|  0|Bill|               0|          [100]|  0|    MS|       ACE|    UC|
|  1|Matt|               1|[500, 250, 100]|  1|   PhD|      ECCS|    UC|
|  2|Zack|               1|     [250, 100]|  1|   PhD|      ECCS|    UC|
+---+----+----------------+---------------+---+------+----------+------+

+----+----+----------------+---------------+---+------+----------+------+
|  id|name|graduate_program|         status| id|degree|department|school|
+----+----+----------------+---------------+---+------+----------+------+
|   0|Bill|               0|          [100]|  0|    MS|       ACE|    UC|
|   1|Matt|               1|[500, 250, 100]|  1|   PhD|      ECCS|    UC|
|   2|Zack|               1|     [250, 100]|  1|   PhD|      ECCS|    UC|
|null|null|            null|           null|

In [34]:
# left semi joins - only include shared key rows
# DOES NOT include anything from right table
# right table only provide the keys for 'filtering' the rows
program.join(person, person['graduate_program'] == program['id'], 'left_semi').show()

+---+------+----------+------+
| id|degree|department|school|
+---+------+----------+------+
|  0|    MS|       ACE|    UC|
|  1|   PhD|      ECCS|    UC|
+---+------+----------+------+



In [35]:
# left anti joins - exlcude shared key rows
# DOES NOT include anything from right table
# right table only provide the keys for 'filtering' the rows
program.join(person, person['graduate_program'] == program['id'], 'left_anti').show()

+---+------+----------+------+
| id|degree|department|school|
+---+------+----------+------+
|  2|    MS|      ECCS|    UC|
+---+------+----------+------+

